### Welcome to PyOMA

Run the cells below with Shift+Enter

In [4]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import matplotlib
import sys
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 

%aimport -sys -logging -matplotlib -matplotlib.pyplot -numpy -pandas - scipy.stats -scipy.stats.qmc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


please modify the path below to point to the code directory and run the cell

In [5]:
sys.path.append('/home/sima9999/git/pyOMA')

from core.PreProcessingTools import PreProcessSignals, GeometryProcessor
from core.StabilDiagram import StabilCalc, StabilCluster, StabilPlot
from core.PostProcessingTools import MergePoSER
from core.PlotMSH import ModeShapePlot

from GUI.JupyterGUI import PlotMSHWeb, StabilGUIWeb, SnappingCursor, OutputWidgetHandler, ConfigGUIWeb

please modify the paths and setting below to point to the project and setup directories and run the cell

In [6]:
project_dir = Path('/home/sima9999/git/pyOMA/tests/files/')
nodes_file = project_dir / 'grid.txt'
lines_file = project_dir / 'lines.txt'
parent_child_file = ''

meas_name = 'measurement_1'
setup_dir = project_dir / meas_name
result_dir = setup_dir

setup_info_file = setup_dir / 'setup_info.txt'
meas_file = setup_dir / (meas_name + '.npy')
oma_conf_file = project_dir / 'varssi_config.txt'
chan_dofs_file = setup_dir / "channel_dofs.txt"

skip_existing=True
save_results=False

PreProcessSignals.load_measurement_file = np.load

In [11]:
display(ConfigGUIWeb(locals()))

In [31]:
geometry_data = GeometryProcessor.load_geometry(
    nodes_file=nodes_file,
    lines_file=lines_file,
    parent_childs_file=parent_child_file)

In [32]:
if not os.path.exists(setup_dir / 
        'prep_data.npz') or not skip_existing:
    prep_data = PreProcessSignals.init_from_config(
        conf_file=setup_info_file,
        meas_file=meas_file,
        chan_dofs_file=chan_dofs_file)

    if save_results:
        prep_data.save_state(result_dir / 'prep_data.npz')
else:
    prep_data = PreProcessSignals.load_state(result_dir / 'prep_data.npz')

INFO:core.PreProcessingTools:Now removing Channel ref_z (no. 5)!


In [33]:
if not os.path.exists(result_dir /
        'modal_data.npz') or not skip_existing:

    modal_data = method.init_from_config(oma_conf_file, prep_data)

    if save_results:
        modal_data.save_state(result_dir / 'modal_data.npz')
else:
    modal_data = method.load_state(
        result_dir / 'modal_data.npz', prep_data)

Assembling toeplitz matrix using pre-computed correlation functions 200 block-columns and 201 block rows
INFO:core.PreProcessingTools:Estimating Correlation Functions (BT) with n_lags=400 and num_blocks=1...
....................................................................................................
Computing state matrices with pinv-based method, with maximum model order 100...
Computing modal parameters...
....................................................................................................


In [26]:
if os.path.exists(result_dir 
                  / 'stabil_data.npz') and skip_existing:
    stabil_calc = StabilCalc.load_state(
        result_dir / 'stabil_data.npz', modal_data, prep_data)
else:
    stabil_calc = StabilCalc(modal_data, prep_data)
stabil_plot = StabilPlot(stabil_calc)

Now loading previous results from  /home/sima9999/git/pyOMA/tests/files/measurement_1/stabil_data.npz


In [27]:
display(StabilGUIWeb(stabil_plot))

In [28]:
if save_results:
    stabil_calc.save_state(result_dir / 'stabil_data.npz')

In [29]:
mode_shape_plot = ModeShapePlot(amplitude=10,
                                prep_data=prep_data,
                                stabil_calc=stabil_calc,
                                geometry_data=geometry_data,
                                modal_data=modal_data)

In [30]:
display(PlotMSHWeb(mode_shape_plot))